In [2]:
from typing import List, Literal, Union
import pandas as pd
import pm4py
from pm4py.objects.ocel.obj import OCEL
from pydantic.main import BaseModel

ocel = pm4py.read_ocel2_sqlite("/home/grkmr/Projects/ocelescope/src/backend/data/event_logs/ContainerLogistics-v2.sqlite")

/home/grkmr/Projects/ocelescope/src/backend/.venv/lib/python3.10/site-packages/pm4py/objects/ocel/importer/sqlite/variants/ocel20.py:75: UserWarning: There are unsatisfied OCEL 2.0 constraints in the given relational database: ['const_23_foreign_key_event_type_specific']
  warnings.warn("There are unsatisfied OCEL 2.0 constraints in the given relational database: "+str(unsatisfied))
/home/grkmr/Projects/ocelescope/src/backend/.venv/lib/python3.10/site-packages/pm4py/objects/log/util/dataframe_utils.py:191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pandas_utils.dataframe_column_string_to_datetime(df[col], format=timest_format, utc=True)
/home/grkmr/Projects/ocelescope/src/backend/.venv/lib/python3.10/site-packages/pm4py/objects/log/u

In [3]:
from pydantic import BaseModel

events = ocel.events

class Date_Distribution_Item(BaseModel):
    date: str
    entity_count: dict[str,int]
    
class EventTimeInfo(BaseModel):
    start_time: str
    end_time: str
    date_distribution: list[Date_Distribution_Item]

In [21]:

timeFrameCount = events.groupby([events[ocel.event_timestamp].dt.date, ocel.event_activity]).size()

In [22]:
timeFrameCount = timeFrameCount.reset_index(name='count')
timeFrameCount

,ocel:timestamp,ocel:activity,count
0,2023-05-22,Register Customer Order,2
1,2023-05-23,Book Vehicles,1
2,2023-05-23,Create Transport Document,1
3,2023-05-23,Order Empty Containers,1
4,2023-05-23,Register Customer Order,2
...,...,...,...
3618,2024-08-21,Order Empty Containers,4
3619,2024-08-21,Pick Up Empty Container,7
3620,2024-08-21,Place in Stock,6
3621,2024-08-21,Reschedule Container,1


In [24]:
date_distribution = []
for date, group in timeFrameCount.groupby(timestamp_column_name):
    row = {
        "date": str(date),
        "entity_count": dict(zip(group[activity_column_name], group["count"]))
    }
    date_distribution.append(Date_Distribution_Item(**row))

start_time = ocel.events[ocel.event_timestamp].min().isoformat(timespec="microseconds")
end_time = ocel.events[ocel.event_timestamp].max().isoformat(timespec="microseconds")
                                                             
EventTimeInfo(end_time=end_time, start_time=start_time, date_distribution=date_distribution)

NameError: name 'timestamp_column_name' is not defined

In [27]:
events = ocel.events
timestamp_column_name = ocel.event_timestamp
activity_column_name = ocel.event_activity

    # Group by date and activity, then count
time_frame_count = events.groupby([events[timestamp_column_name].dt.date, activity_column_name]).size().reset_index(name='count')

# Build distribution per date
date_distribution = []
for date, group in time_frame_count.groupby(timestamp_column_name):
    row = {
        "date": str(date),
        "entity_count": dict(zip(group[activity_column_name], group["count"]))
    }
    date_distribution.append(Date_Distribution_Item(**row))

# Get start and end time of events
start_time = events[timestamp_column_name].min().isoformat(timespec="microseconds")
end_time = events[timestamp_column_name].max().isoformat(timespec="microseconds")

In [28]:
date_distribution

[Date_Distribution_Item(date='2023-05-22', entity_count={'Register Customer Order': 2}),
 Date_Distribution_Item(date='2023-05-23', entity_count={'Book Vehicles': 1, 'Create Transport Document': 1, 'Order Empty Containers': 1, 'Register Customer Order': 2}),
 Date_Distribution_Item(date='2023-05-24', entity_count={'Collect Goods': 1, 'Pick Up Empty Container': 1, 'Register Customer Order': 2}),
 Date_Distribution_Item(date='2023-05-25', entity_count={'Book Vehicles': 1, 'Collect Goods': 11, 'Create Transport Document': 1, 'Drive to Terminal': 2, 'Load Truck': 12, 'Pick Up Empty Container': 1, 'Place in Stock': 2, 'Register Customer Order': 2, 'Weigh': 2}),
 Date_Distribution_Item(date='2023-05-26', entity_count={'Book Vehicles': 3, 'Create Transport Document': 4, 'Order Empty Containers': 1, 'Register Customer Order': 1}),
 Date_Distribution_Item(date='2023-05-27', entity_count={'Collect Goods': 1, 'Pick Up Empty Container': 1}),
 Date_Distribution_Item(date='2023-05-29', entity_count=